In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pathlib import Path

In [12]:
#For different groups change the text between /Raw Data and /Collar in MainPath and /Processed Data and /ACC in SavePath
#Different Groups -  ZU_2021_1, ZU_2021_2, NQ_2021_1 and RW_2021_1

MainPath = '/media/amlan/Data/Thesis Data/Raw Data/ZU_2021_1/COLLAR/GPS/'
SavePath = '/media/amlan/Data/Thesis Data/Processed Data/ZU_2021_1/ACC/'

os.makedirs(SavePath, exist_ok=True)

paths = os.listdir(MainPath)
paths = [i for i in paths if 'DS_Store' not in i]


In [13]:
print(paths)

['ZU_VZUF051_RRTB_Axy010_20210516-20210524', 'ZU_VZUF052_RRRT_Axy018_20210516-20210524', 'ZU_VZUF054_RRRS_Axy003_20210515-20210516', 'ZU_VZUF054_RRRS_Axy019_20210517-20210524', 'ZU_VZUM053_RRLT_Axy001_20210518-20210524', 'ZU_VZUM056_LTRT_Axy004_20210516-20210518', 'ZU_VZUM057_LTTB_Axy015_20210519-20210524', 'ZU_VZUM057_LTTB_Axy019_20210516-20210516', 'ZU_VZUM059_LTLS_Axy011_20210516-20210523']


In [14]:
freq=10 #Enter the known fequency for the particular group. NQ1 and ZU2 are 50 Hz, RW and ZU1 are 10 Hz.

def clean_leading_zeros(df):
    
    if pd.to_datetime(df.loc[1,'Timestamp']) - pd.to_datetime(df.loc[0,'Timestamp']) >= pd.Timedelta(1, "s") :
        Leading_Zeros = df.loc[~(df.drop(['Timestamp'],axis=1)==0).all(axis=1)]
        if Leading_Zeros.empty:
            return Leading_Zeros
        else:
            print('There are leading zeros till ',Leading_Zeros.index[0])
            return df.loc[Leading_Zeros.index[0]::,:]
    else:
        return df

def load_ACC_files(x,MainPath):
    #File = 'NQ_VNQF014_SHLT_Axy_002_20210811-20210817/NQ_VNQF014_SHLT_Axy_002_20210811-20210817.csv'
    
    File = str(x) + '/' + str(x) + '.csv' 
    Data = pd.read_csv(MainPath + File,usecols = ['Timestamp','X','Y','Z'])
    print('--------------------------------------------------')
    print('\nFile: {} with Dimensions {}'.format(x,Data.shape))
    print('Data in the file:')
    print(Data.head(3))
    
    Data = clean_leading_zeros(Data)
    print('\nData After Cleaning Leading Zeros:')
    print(Data.head(3)) 
    
    print('Dimensions after removing leading empty rows: ',Data.shape)
    Data = Data.reset_index(drop=True)
          
    return Data    
    
def feature_extraction_ACC(Values,Unique_Time,freq):
    
    Norm = np.sqrt(np.sum(np.square(Values),axis=1))
    t = pd.to_datetime(Data.loc[0:Unique_Time*freq-1:freq,'Timestamp'],format="%d/%m/%Y %H:%M:%S.%f")
    vedba = np.std(Norm.reshape(Unique_Time,freq),axis=1)
    means = np.mean(Values.reshape((Unique_Time,freq,3)),axis=1)
    maxes = np.max(Values.reshape((Unique_Time,freq,3)),axis=1)
    mins = np.min(Values.reshape((Unique_Time,freq,3)),axis=1)
    std = np.std(Values.reshape((Unique_Time,freq,3)),axis=1)
    
    return t,vedba,means,std,maxes,mins


    
for y in paths:
    
    
    Data = load_ACC_files(y,MainPath)


    Unique_Time= Data.shape[0]//freq
    print('\nUnique Times: ',Unique_Time)


    Values = Data.iloc[0:Unique_Time*freq,1:4].to_numpy()
    print('Dimension of ACC Values After Considering Unique Times: ',Values.shape)
    
    t,vedba,means,std,maxes,mins = feature_extraction_ACC(Values,Unique_Time,freq)


    Features_Data = pd.DataFrame(columns=['Timestamp','X_Mean','Y_Mean','Z_Mean','X_Std','Y_Std','Z_Std','X_Max','Y_Max','Z_Max','X_Min','Y_Min','Z_Min','VeDBA'])
    Features_Data['Timestamp'] = t
    Features_Data['VeDBA'] = vedba
    Features_Data.iloc[:,1:4] = means
    Features_Data.iloc[:,4:7] = std
    Features_Data.iloc[:,7:10] = maxes
    Features_Data.iloc[:,10:13] = mins
    
    Features_Data = Features_Data.reset_index(drop=True)
    
    Features_Data.to_csv(SavePath + y +'.csv')


--------------------------------------------------

File: ZU_VZUF051_RRTB_Axy010_20210516-20210524 with Dimensions (8403291, 4)
Data in the file:
                 Timestamp      X      Y      Z
0  14/05/2021 16:15:24.000 -1.000 -0.438 -0.125
1  14/05/2021 16:15:24.100 -0.888 -0.550 -0.013
2  14/05/2021 16:15:24.200 -0.925 -0.563  0.000

Data After Cleaning Leading Zeros:
                 Timestamp      X      Y      Z
0  14/05/2021 16:15:24.000 -1.000 -0.438 -0.125
1  14/05/2021 16:15:24.100 -0.888 -0.550 -0.013
2  14/05/2021 16:15:24.200 -0.925 -0.563  0.000
Dimensions after removing leading empty rows:  (8403291, 4)

Unique Times:  840329
Dimension of ACC Values After Considering Unique Times:  (8403290, 3)
--------------------------------------------------

File: ZU_VZUF052_RRRT_Axy018_20210516-20210524 with Dimensions (7790771, 4)
Data in the file:
                 Timestamp    X    Y    Z
0  12/05/2021 14:20:20.000  0.0  0.0  0.0
1  12/05/2021 14:20:49.000  0.0  0.0  0.0
2  12/05/

In [62]:
Data

,Timestamp,X,Y,Z
0,18/02/2020 16:02:01.000,0.0,0.0,0.0
1,18/02/2020 16:02:30.000,0.0,0.0,0.0
2,18/02/2020 16:03:00.000,0.0,0.0,0.0
3,18/02/2020 16:03:30.000,0.0,0.0,0.0
4,18/02/2020 16:04:00.000,0.0,0.0,0.0
...,...,...,...,...
506,18/02/2020 19:52:30.900,0.0,0.0,0.0
507,18/02/2020 19:52:30.920,0.0,0.0,0.0
508,18/02/2020 19:52:30.940,0.0,0.0,0.0
509,18/02/2020 19:52:30.960,0.0,0.0,0.0


In [64]:
df = Data
df.loc[~(df.drop(['Timestamp'],axis=1)==0).all(axis=1)]

,Timestamp,X,Y,Z


In [26]:
(Features_Data['VeDBA'] == 0).sum()

18440

In [42]:

fig.savefig('ZU1_2021.png',dpi=400)

In [14]:
def clean_leading_zeros(df):
    
    if pd.to_datetime(df.loc[1,'Timestamp']) - pd.to_datetime(df.loc[0,'Timestamp']) >= pd.Timedelta(1, "s") :
        Leading_Zeros = df.loc[~(df.drop(['Timestamp'],axis=1)==0).all(axis=1)].index[0]
        print('There are leading zeros till ',Leading_Zeros)
        return df.loc[Leading_Zeros::,:]
    else:
        return df

y = paths[1]
freq = 10
File = str(y) + '/' + str(y) + '.csv' 
Data = pd.read_csv(MainPath + File,usecols = ['Timestamp','X','Y','Z']) 
print('\nFile: ', y)
print('Dimensions: ',Data.shape)
print(Data.head(3))

Data = clean_leading_zeros(Data)
print(Data.head(3))


print('Dimensions after removing leading empty rows: ',Data.shape)
#print(Data.reset_index(drop=True).head())
Data = Data.reset_index(drop=True)

    
Unique_Time= Data.shape[0]//freq
print('Unique Times: ',Unique_Time)


Values = Data.iloc[0:Unique_Time*freq,1:4].to_numpy()
print('Dimension After Considering Unique Times: ',Values.shape)

Norm = np.sqrt(np.sum(np.square(Values),axis=1))
Features_Data = pd.DataFrame(columns=['Timestamp','X_Mean','Y_Mean','Z_Mean','X_Max','Y_Max','Z_Max','X_Min','Y_Min','Z_Min','VeDBA'])
t,vedba,means, = [],[],[]
    #x_max,x_min,y_max,y_min,z_max,z_min = [],[],[],[],[],[]
#for i in range(Unique_Time):
#        t.append(pd.to_datetime(Data.loc[freq*i,'Timestamp'],format="%d/%m/%Y %H:%M:%S.%f"))
 #       vedba.append(np.std(Norm[freq*i:freq*(i+1)]))
 #       means.append(np.mean(Values[freq*i:freq*(i+1),:],axis=0))
        #maxes = np.max(Values[freq*i:freq*(i+1),:],axis=0)
        #mins = np.min(Values[freq*i:freq*(i+1),:],axis=0)



File:  ZU_VZUF052_RRRT_Axy018_20210516-20210524
Dimensions:  (7790771, 4)
                 Timestamp    X    Y    Z
0  12/05/2021 14:20:20.000  0.0  0.0  0.0
1  12/05/2021 14:20:49.000  0.0  0.0  0.0
2  12/05/2021 14:21:19.000  0.0  0.0  0.0
There are leading zeros till  8640
                    Timestamp      X      Y      Z
8640  15/05/2021 14:20:19.000  0.000 -0.938  0.063
8641  15/05/2021 14:20:19.100  0.050 -0.988  0.013
8642  15/05/2021 14:20:19.200  0.063 -0.963 -0.038
Dimensions after removing leading empty rows:  (7782131, 4)
Unique Times:  778213
Dimension After Considering Unique Times:  (7782130, 3)


In [20]:
#Slower Looping Version using lists 
t,vedba,means, = [],[],[]
Maxes,Mins = [],[]
for i in range(Unique_Time):
        t.append(pd.to_datetime(Data.loc[freq*i,'Timestamp'],format="%d/%m/%Y %H:%M:%S.%f"))
        vedba.append(np.std(Norm[freq*i:freq*(i+1)]))
        means.append(np.mean(Values[freq*i:freq*(i+1),:],axis=0))
        Maxes.append(np.max(Values[freq*i:freq*(i+1),:],axis=0))
        Mins.append(np.min(Values[freq*i:freq*(i+1),:],axis=0))

In [21]:
t[0:5],t[-10::]

([Timestamp('2021-05-15 14:20:19'),
  Timestamp('2021-05-15 14:20:20'),
  Timestamp('2021-05-15 14:20:21'),
  Timestamp('2021-05-15 14:20:22'),
  Timestamp('2021-05-15 14:20:23')],
 [Timestamp('2021-05-24 14:30:22'),
  Timestamp('2021-05-24 14:30:23'),
  Timestamp('2021-05-24 14:30:24'),
  Timestamp('2021-05-24 14:30:25'),
  Timestamp('2021-05-24 14:30:26'),
  Timestamp('2021-05-24 14:30:27'),
  Timestamp('2021-05-24 14:30:28'),
  Timestamp('2021-05-24 14:30:29'),
  Timestamp('2021-05-24 14:30:30'),
  Timestamp('2021-05-24 14:30:31')])

In [24]:
T= pd.to_datetime(Data.loc[0:Unique_Time*freq-1:freq,'Timestamp'],format="%d/%m/%Y %H:%M:%S.%f")
T[0:5],T[-10::]

(0    2021-05-15 14:20:19
 10   2021-05-15 14:20:20
 20   2021-05-15 14:20:21
 30   2021-05-15 14:20:22
 40   2021-05-15 14:20:23
 Name: Timestamp, dtype: datetime64[ns],
 7782030   2021-05-24 14:30:22
 7782040   2021-05-24 14:30:23
 7782050   2021-05-24 14:30:24
 7782060   2021-05-24 14:30:25
 7782070   2021-05-24 14:30:26
 7782080   2021-05-24 14:30:27
 7782090   2021-05-24 14:30:28
 7782100   2021-05-24 14:30:29
 7782110   2021-05-24 14:30:30
 7782120   2021-05-24 14:30:31
 Name: Timestamp, dtype: datetime64[ns])

In [25]:
vedba[0:5], vedba[-5::]

([0.14731904623265418,
  0.18768463229058646,
  0.18435571156795733,
  0.2286249352913698,
  0.16249118130245607],
 [0.04776558218394699,
  0.023159757320851744,
  0.008269332792060735,
  0.0180106010264387,
  0.021037291735867998])

In [26]:
V = Norm.reshape((Unique_Time,freq)).std(axis=1)

V[0:5], V[-5::]

(array([0.14731905, 0.18768463, 0.18435571, 0.22862494, 0.16249118]),
 array([0.04776558, 0.02315976, 0.00826933, 0.0180106 , 0.02103729]))

In [27]:
means[0:5], means[-5::]

([array([-0.0176, -1.0291,  0.1439]),
  array([ 0.0064, -1.0003,  0.2003]),
  array([-0.1689, -0.8189,  0.419 ]),
  array([ 0.0752, -1.0189,  0.1315]),
  array([-0.0252, -1.0066,  0.1628])],
 [array([ 0.4192, -0.0754,  0.8066]),
  array([ 0.3504, -0.0692,  0.8751]),
  array([ 0.4317, -0.0441,  0.813 ]),
  array([ 0.3565, -0.0629,  0.8813]),
  array([ 0.4254, -0.0063,  0.8254])])

In [29]:
M= Values.reshape((Unique_Time,freq,3)).mean(axis=1)
M[0:5], M[-5::]

(array([[-0.0176, -1.0291,  0.1439],
        [ 0.0064, -1.0003,  0.2003],
        [-0.1689, -0.8189,  0.419 ],
        [ 0.0752, -1.0189,  0.1315],
        [-0.0252, -1.0066,  0.1628]]),
 array([[ 0.4192, -0.0754,  0.8066],
        [ 0.3504, -0.0692,  0.8751],
        [ 0.4317, -0.0441,  0.813 ],
        [ 0.3565, -0.0629,  0.8813],
        [ 0.4254, -0.0063,  0.8254]]))

In [30]:
Maxes[0:5],Maxes[-5::]

([array([ 0.063, -0.938,  0.45 ]),
  array([ 0.25 , -0.813,  0.563]),
  array([ 0.188, -0.375,  0.875]),
  array([ 0.25 , -0.75 ,  0.375]),
  array([ 0.125, -0.875,  0.563])],
 [array([ 0.438, -0.063,  0.875]),
  array([ 0.438, -0.063,  0.938]),
  array([0.438, 0.   , 0.813]),
  array([0.438, 0.   , 0.938]),
  array([0.438, 0.   , 0.875])])

In [31]:
Maax= Values.reshape((Unique_Time,freq,3)).max(axis=1)
Maax[0:5], Maax[-5::]

(array([[ 0.063, -0.938,  0.45 ],
        [ 0.25 , -0.813,  0.563],
        [ 0.188, -0.375,  0.875],
        [ 0.25 , -0.75 ,  0.375],
        [ 0.125, -0.875,  0.563]]),
 array([[ 0.438, -0.063,  0.875],
        [ 0.438, -0.063,  0.938],
        [ 0.438,  0.   ,  0.813],
        [ 0.438,  0.   ,  0.938],
        [ 0.438,  0.   ,  0.875]]))

In [32]:
Mins[0:5],Mins[-5::]

([array([-0.1  , -1.375, -0.038]),
  array([-0.125, -1.375,  0.   ]),
  array([-0.438, -1.375, -0.125]),
  array([-0.125, -1.438, -0.125]),
  array([-0.125, -1.375,  0.   ])],
 [array([ 0.313, -0.125,  0.625]),
  array([ 0.313, -0.125,  0.813]),
  array([ 0.375, -0.063,  0.813]),
  array([ 0.313, -0.125,  0.875]),
  array([ 0.375, -0.063,  0.813])])

In [34]:
Miin= np.min(Values.reshape((Unique_Time,freq,3)),axis=1)
Miin[0:5], Miin[-5::]

(array([[-0.1  , -1.375, -0.038],
        [-0.125, -1.375,  0.   ],
        [-0.438, -1.375, -0.125],
        [-0.125, -1.438, -0.125],
        [-0.125, -1.375,  0.   ]]),
 array([[ 0.313, -0.125,  0.625],
        [ 0.313, -0.125,  0.813],
        [ 0.375, -0.063,  0.813],
        [ 0.313, -0.125,  0.875],
        [ 0.375, -0.063,  0.813]]))

In [71]:
Values

array([[ 0.   , -0.938,  0.063],
       [ 0.05 , -0.988,  0.013],
       [ 0.063, -0.963, -0.038],
       ...,
       [ 0.438,  0.   ,  0.813],
       [ 0.438,  0.   ,  0.813],
       [ 0.438,  0.   ,  0.813]])

In [148]:
#Vectorize features
#pd.to_datetime(Data.loc[::freq,'Timestamp'],format="%d/%m/%Y %H:%M:%S.%f")
#Norm.reshape((Unique_Time,freq)).std(axis=1)
#Values.reshape((Unique_Time,freq,3)).mean(axis=1)
#Values.reshape((Unique_Time,freq,3)).max(axis=1)
#Values.reshape((Unique_Time,freq,3)).min(axis=1)

0         2021-05-14 16:15:24.000
10        2021-05-14 16:15:25.000
20        2021-05-14 16:15:26.000
30        2021-05-14 16:15:27.000
40        2021-05-14 16:15:28.000
                    ...          
8403210   2021-05-24 09:40:48.400
8403220   2021-05-24 09:40:49.400
8403230   2021-05-24 09:40:50.400
8403240   2021-05-24 09:40:51.400
8403250   2021-05-24 09:40:52.400
Name: Timestamp, Length: 840326, dtype: datetime64[ns]

In [51]:
MainPath = '/media/amlan/Data/Thesis Data/Raw Data/ZU_2021_1/COLLAR/GPS/'
os.makedirs(SavePath, exist_ok=True)  
paths = os.listdir(MainPath)
paths = [i for i in paths if 'DS_Store' not in i]
y = paths[0]

In [60]:
def clean_leading_zeros(df):
    
    if pd.to_datetime(df.loc[1,'Timestamp']) - pd.to_datetime(df.loc[0,'Timestamp']) >= pd.Timedelta(1, "s") :
        Leading_Zeros = df.loc[~(df.drop(['Timestamp'],axis=1)==0).all(axis=1)].index[0]
        print('There are leading zeros till ',Leading_Zeros)
        return df.loc[Leading_Zeros::,:]
    else:
        return df

y = paths[0]
freq = 10
File = str(y) + '/' + str(y) + '.csv' 
Data = pd.read_csv(MainPath + File,usecols = ['Timestamp','X','Y','Z']) 
print('\nFile: ', y)
print('Dimensions: ',Data.shape)
print(Data.head())

Data = clean_leading_zeros(Data)
print(Data.head())


File:  ZU_VZUF051_RRTB_Axy010_20210516-20210524
Dimensions:  (8403291, 4)
                 Timestamp      X      Y      Z
0  14/05/2021 16:15:24.000 -1.000 -0.438 -0.125
1  14/05/2021 16:15:24.100 -0.888 -0.550 -0.013
2  14/05/2021 16:15:24.200 -0.925 -0.563  0.000
3  14/05/2021 16:15:24.300 -0.894 -0.563  0.044
4  14/05/2021 16:15:24.400 -0.875 -0.563  0.025
                 Timestamp      X      Y      Z
0  14/05/2021 16:15:24.000 -1.000 -0.438 -0.125
1  14/05/2021 16:15:24.100 -0.888 -0.550 -0.013
2  14/05/2021 16:15:24.200 -0.925 -0.563  0.000
3  14/05/2021 16:15:24.300 -0.894 -0.563  0.044
4  14/05/2021 16:15:24.400 -0.875 -0.563  0.025
